# Implementação do SPHINCS+

In [1]:
# Imports
import hashlib
import random
import math
import os

## Classe Connect

    Nesta classe address

In [2]:
class Connect:
    # Types
    WOTSHASH = 0
    WOTSPK = 1
    TREE = 2
    FORSTREE = 3
    FORSROOTS = 4
        
    def __init__(self):
        self.layer = 0
        self.treeAddress = 0
        self.type = 0
        self.one = 0
        self.two = 0
        self.three = 0
        
    def copy(self):
        conn = Connect()
        conn.layer = self.layer
        conn.treeAddress = self.treeAddress
        conn.type = self.type
        conn.one = self.one
        conn.two = self.two
        conn.three = self.three
        
        return conn
    
    def toBin(self): # Not working
        conn = bytes(self.layer)
        conn += bytes(self.treeAddress.digits(base=256))
        
        conn += bytes(self.type.digits(base=256))
        conn += bytes(self.one)
        conn += bytes(self.two)
        conn += bytes(self.three.digits(base=256))
        
        return conn
        
    def setType(self, val):
        self.type = val
        
        self.one = 0
        self.two = 0
        self.three = 0
        
    def setLayerAdrs(self, conn):
        self.layer = conn
        
    def setTreeAdrs(self, conn):
        self.treeAdress = conn
        
    def setPairAdrs(self, conn):
        self.one = conn
        
    def getPairAdrs(self):
        return self.one
        
    def setChainAdrs(self, conn):
        self.two = conn
        
    def setHashAdrs(self, conn):
        self.three = conn
        
    def setTreeHeight(self, height):
        self.two = height
        
    def getTreeHeight(self):
        return self.two
        
    def setTreeIndex(self, index):
        self.three = index
        
    def getTreeIndex(self):
        return self.three
    
# Convertion to base W
def baseW(x, w, lenth):
    xin = 0
    xout = 0
    total = 0
    bits = 0
    bw = []
    
    for consumed in range(0, lenth):
        if bits == 0:
            total = x[xin]
            xin += 1
            bits += 8
        bits -= math.floor(math.log(w,2))
        bw.append((total >> bits) % w)
        xout += 1
        
    return bw

## Parâmetros

In [3]:
# Parameters

_randomize = True

N = 16 # Security parameter
W = 4  # Winternitz parameter (usualy 4, 16 or 256)
H = 64 # HyperTree height
D = 8  # HyperTree layers
K = 10 # Fors trees
A = 15 # Fors tree height
T = 2 ** A # Fors tree leaves

L1 = math.ceil(8 * N /math.log(W,2)) # len 1
L2 = math.floor(math.log(L1 * (W - 1),2) / math.log(W,2)) + 1 # len 2
L0 = L1 + L2 # len 0
HPrime = H // D

## Tweakable Hash Funcions

    Nesta secção é feita são definidas as funções de hash. Este tipo de funções de hash são recomendados para o contexto definido e tomam o lugar de nonces em termos de segurança. Para além disso, permitem serem chamadas em cada par de chaves do SPHINCS+ e em cada posição da estrutura da árvore virtual do SPHINCS+ de forma independente entre si.

In [4]:
# Hash (Sha256 based)
def hash(seed, conn: Connect, value, digestSize):
    h = hashlib.sha256()
    
    h.update(seed)
    h.update(conn.toBin())
    h.update(value)
    
    hashed = h.digest()[:digestSize]
    
    return hashed

# Pseudorandom Function
def prf(skSeed, conn, digestSize):
    random.seed(int.from_bytes(skSeed + conn.toBin(),"big"))
    return random.randint(0, 256 ** digestSize - 1).to_bytes(digestSize, byteorder='big')

# Message Hash
def hashMsg(r, pkSeed, pkRoot, value, digestSize):
    h = hashlib.sha256()
    
    h.update(r)
    h.update(pkSeed)
    h.update(pkRoot)
    h.update(value)
    
    hashed = h.digest()[:digestSize]
    
    i = 0
    while len(hashed) < digestSize:
        i += 1
        h = hashlib.sha256()
        
        h.update(r)
        h.update(pkSeed)
        h.update(pkRoot)
        h.update(value)
        h.update(bytes([i]))
        
        hashed += h.digest()[:digestSize - len(hashed)]
        
    return hashed

# Message Pseudorandom Function
def prfMsg(skSeed, opt, m, digestSize):
    random.seed(int.from_bytes(skSeed + opt + hashMsg(b'0', b'0', b'0', m, digestSize * 2),"big"))
    return random.randint(0, 256 ** digestSize - 1).to_bytes(digestSize, byteorder='big')

## Wots+

    É um esquema de assinatura One-Time signature, isto é, uma chave privada deve ser utilizada para assinar apenas uma
    mensagem.
### chain:
    Método responsável por iterar o valor de x s steps. Utiliza o endereço de Connect e  a public seed para a operação.

In [5]:
# Chaining
def chain(x, i, s, pkSeed, conn: Connect):
    if s == 0:
        return bytes(x)
    
    if (i + s) > (W - 1):
        return -1
    tmp = chain(x, i, s - 1, pkSeed, conn)
    
    conn.setHashAdrs(i + s - 1)
    tmp = hash(pkSeed, conn, tmp, N)
    
    return tmp

# Public Key
def wotsGenPk(skSeed, pkSeed, conn: Connect):
    
    wotsPkConn = conn.copy()
    tmp = bytes()
    
    for i in range(0, L0):
        # Secret Key
        conn.setChainAdrs(i)
        conn.setHashAdrs(0)
        sk = prf(skSeed, conn.copy(), N)
        
        tmp += bytes(chain(sk, 0, W - 1, pkSeed, conn.copy()))
        
    wotsPkConn.setType(Connect.WOTSPK)
    wotsPkConn.setPairAdrs(conn.getPairAdrs())
    
    pk = hash(pkSeed, wotsPkConn, tmp, N)
    
    return pk

# Signature
def wotsSign(m, skSeed, pkSeed, conn):
    
    csum = 0
    
    msg = baseW(m, W, L1) # base W
    
    # checksum calculation
    for i in range(0, L1):
        csum += W -1 - msg[i] 
    
    # checksum to base W
    padding = (L2 * math.floor(math.log(W,2))) % 8 if (L2 * math.floor(math.log(W,2))) % 8 != 0 else 8

    csum = csum << (8 - padding)
    csumb = int(csum).to_bytes(math.ceil((L2 * math.floor(math.log(W,2))) / 8), byteorder='big')
    csumw = baseW(csumb, W, L2)
    msg += csumw
    
    sig = []
    for i in range(0, L0):
        conn.setChainAdrs(i)
        conn.setHashAdrs(0)
        sk = prf(skSeed, conn.copy(), N)
        sig += [chain(sk,0, msg[i], pkSeed, conn.copy())]
        
    return sig

# Pk from Signature
def wotsPkSig(sig, m, pkSeed, conn: Connect):
    csum = 0
    wotsPkConn = conn.copy()
    
    msg = baseW(m, W, L1)
    
    for i in range(0, L1):
        csum += W - 1 - msg[i]
    
    padding = (L2 * math.floor(math.log(W,2))) % 8 if (L2 * math.floor(math.log(W,2))) % 8 != 0 else 8

    csum = csum << (8 - padding)
    csumb = int(csum).to_bytes(math.ceil((L2 * math.floor(math.log(W,2))) / 8), byteorder='big')
    csumw = baseW(csumb, W, L2)
    msg += csumw
    
    tmp = bytes()
    for i in range(0, L0):
        conn.setChainAdrs(i)
        tmp += chain(sig[i], msg[i], W - 1 - msg[i], pkSeed, conn.copy())
        
    wotsPkConn.setType(Connect.WOTSPK)
    wotsPkConn.setPairAdrs(conn.getPairAdrs())
    
    pkSig = hash(pkSeed, wotsPkConn, tmp, N)
    
    return pkSig


## Hypertree (XMSS)

    Para a implementação do Hypertree são primeiro definidos métodos do tipo single tree, neste caso XMSS. Desta froma conseguimos combinar o WOTS+ com as árvores binárias de hash o que resulta numa vesão com input fixo.
    O XMSS (eXtended Markle Signature Scheme) permite assinar um número fixo de mensagens baseado no esquema de assinaturas Merkle. Cada nodo da àrvore é um valor de n-bytes representado pela teakable hash da concatenação dos nodos dos filhos. 
    Nese caso de implementação do  SPHINCS+ apeans é utilizada a secret seed para gerar todas as secrets keys do WOTS+.

In [6]:
# Return the root node (n-byte)
def treeHash(skSeed, s, z, pkSeed, conn: Connect):
    if s % (1 << z) != 0:
        return -1
    
    stack = []
    
    for i in range(0, 2 ** z):
        conn.setType(Connect.WOTSHASH)
        conn.setPairAdrs(s + i)
        node = wotsGenPk(skSeed, pkSeed, conn.copy())
        
        conn.setType(Connect.TREE)
        conn.setTreeHeight(1)
        conn.setTreeIndex(s+i)
        
        if len(stack) > 0:
            while stack[len(stack)-1]['height'] == conn.getTreeHeight():
                conn.setTreeIndex((conn.getTreeIndex() - 1) // 2)
                node = hash(pkSeed, conn.copy(), stack.pop()['node'] + node, N)
                conn.setTreeHeight(conn.getTreeHeight() + 1)
                
                if len (stack) <= 0:
                    break
                    
        stack.append({'node': node, 'height': conn.getTreeHeight()})
        
    return stack.pop()['node']
                
# Public Key
def xmssPkGen(skSeed, publicKey, conn: Connect):
    pk = treeHash(skSeed, 0, HPrime, publicKey, conn.copy())
    
    return pk

# Sign
def xmssSign(m, skSeed, idx, pkSeed, conn):
    auth = []
    
    # Authentication path
    for j in range(0, HPrime):
        ki = math.floor(idx // 2 ** j)
        if ki % 2 == 1: # XOR idx / 2**j with 1
            ki -= 1
        else:
            ki += 1
            
        auth += [treeHash(skSeed, ki * 2 **  j, j, pkSeed, conn.copy())]
    
    conn.setType(Connect.WOTSHASH)
    conn.setPairAdrs(idx)
    
    sig = wotsSign(m, skSeed, pkSeed, conn.copy())
    sigXmss = sig + auth
    
    
    return sigXmss

# Pk from Signature
def xmssPkSig(idx, sigXmss, m, pkSeed, conn):
    
    # WOTS+ pk from WOTS+ signature
    conn.setType(Connect.WOTSHASH)
    conn.setPairAdrs(idx)
    sig = sigWotsSigXmss(sigXmss)
    auth = authSigXmss(sigXmss)
    
    nodo0 = wotsPkSig(sig, m, pkSeed, conn.copy())
    nodo1 = 0
    
    # Root from WOTS+ pk and authentication
    conn.setType(Connect.TREE)
    conn.setTreeIndex(idx)
    for i in range(0, HPrime):
        conn.setTreeHeight(i + 1)
        
        if math.floor(idx / 2 ** i) % 2 == 0:
            conn.setTreeIndex(conn.getTreeIndex() // 2)
            nodo1 = hash(pkSeed, conn.copy(), nodo0 + auth[i], N)
        else:
            conn.setTreeIndex((conn.getTreeIndex() - 1) // 2)
            nodo1 = hash(pkSeed, conn.copy(), auth[i] + nodo0, N)
            
        nodo0 = nodo1
    
    return nodo0
    

## Hypertree HT

    Nesta secção é feita a implementação de uma variante do XMSS. É implementada uma tree of trees (árvore de árvores), basicamente é uma árvore com várias camadas de árvores. Sendo utilizadas as camadas superiores e intermédias para assinar as chaves públicas.

In [7]:
# Public Key (unic superior layer XMSS tree root)
def htGenPk(skSeed, pkSeed):
    conn = Connect()
    conn.setLayerAdrs(D - 1)
    conn.setTreeAdrs(0)
    root = xmssPkGen(skSeed, pkSeed, conn.copy())
    
    return root

# Signature
def htSing(m, skSeed, pkSeed, indexTree, indexLeaf):
    # inicialization
    conn = Connect()
    conn.setLayerAdrs(0)
    conn.setTreeAdrs(indexTree)
    
    # sign
    sigTmp = xmssSign(m, skSeed, indexLeaf, pkSeed, conn.copy())
    sigHt = sigTmp
    root = xmssPkSig(indexLeaf, sigTmp, m, pkSeed, conn.copy())
    
    for j in range(1, D):
        indexLeaf = indexTree % 2 ** HPrime
        indexTree = indexTree >> HPrime
        
        conn.setLayerAdrs(j)
        conn.setTreeAdrs(indexTree)
        
        sigTmp = xmssSign(root, skSeed, indexLeaf, pkSeed, conn.copy())
        sigHt = sigHt + sigTmp
        
        if j < D - 1:
            root = xmssPkSig(indexLeaf, sigTmp, root, pkSeed, conn.copy())
            
    return sigHt

# Verify Signature
def htVerify(m, sigHt, pkSeed, indexTree, indexLeaf, publicKeyHt):
    
    conn = Connect()
    
    # verification
    sigsXmss = sigWotsSigXmss(sigHt)
    sigTmp = sigsXmss[0]
    
    conn.setLayerAdrs(0)
    conn.setTreeAdrs(indexTree)
    node = xmssPkSig(indexLeaf, sigTmp, m, pkSeed, conn)
    
    for j in range(1, D):
        indexLeaf = indexTree % 2 ** HPrime
        indexTree = indexTree >> HPrime
        
        sigTmp = sigsXmss[j]
        
        conn.setLayerAdrs(j)
        conn.setTreeAdrs(indexTree)
        
        node = xmssPkSig(indexLeaf, sigTmp, node, pkSeed, conn)
        
    if node == publicKeyHt:
        return True
    else:
        return False


## FORS - Forest of Random Subsets

    A Hypertree HT é utilizada para assinar as chaves públicas das instancias FORS e não as mensagens.
    São as instancias FORS que são responsaveis por assinar as mensagens.
    Uma mensagem privada é formada por kt strings de n-bytes aleatórias, agrupadas em k sets.
    
    A verificação das chaves publicas é feita implicitamente, pois existe apenas um método para calcular uma chave publica candidata.

In [8]:
# Private Key
def forsGenSk(skSeed, conn: Connect, idx):
    conn.setTreeHeight(0)
    conn.setTreeIndex(idx)
    sk = prf(skSeed, conn.copy(), N)
    
    return sk

# TreeHash (changes: way the leaf calculation is done and address management)
def forsTreehash(skSeed, s, z, pkSeed, conn):
    if s % (1 << z) != 0:
        return -1
    
    stack = []
    
    for i in range(0, 2 ** z):
        conn.setTreeHeight(0)
        conn.setTreeIndex(s+i)
        sk = prf(skSeed, conn.copy(), N)
        node = hash(pkSeed, conn.copy(), sk, N)
        
        conn.setTreeHeight(1)
        conn.setTreeIndex(s+i)
        
        if len(stack) > 0:
            while stack[len(stack)-1]['height'] == conn.getTreeHeight():
                conn.setTreeIndex((conn.getTreeIndex() - 1) // 2)
                node = hash(pkSeed, conn.copy(), stack.pop()['node'] + node, N)
                conn.setTreeHeight(conn.getTreeHeight() + 1)
                
                if len (stack) <= 0:
                    break
                    
        stack.append({'node': node, 'height': conn.getTreeHeight()})
        
    return stack.pop()['node']

# Public Key
def forsGenPk(skSeed, pkSeed, conn: Connect):
    
    forsConn = conn.copy()
    
    root = bytes()
    for i in range(0, K):
        root += forsTreehash(skSeed, i * T, A, pkSeed, conn)
        
    forsConn.setType(Connect.FORSROOTS)
    forsConn.setPairAdrs(conn.getPairAdrs())
    pk = hash(pkSeed, forsConn, root, N)
    
    return pk

# Signature
def forsSign(m, skSeed, pkSeed, conn):
    mInt = int.from_bytes(m, 'big')
    sigFors = []
    
    # Signature elements calculation
    for i in range(0, K):
        
        # next index
        idx = (mInt >> (K -1 -i) * A) % T
        
        # private key element
        conn.setTreeHeight(0)
        conn.setTreeIndex(i * T + idx)
        sigFors += [prf(skSeed, conn.copy(), N)]
        
        auth = []
        
        # authentication path
        for j in range(0, A):
            s = math.floor(idx // 2 ** j)
            if s % 2 == 1:
                s -= 1
            else:
                s += 1
            
            auth += [forsTreehash(skSeed, i * T + s * 2 ** j, j, pkSeed, conn.copy())]
            
        sigFors += auth
    
    return sigFors

# Pk from Signature
def forsPkSig(sigFors, m, pkSeed, conn:Connect):
    
    mInt = int.from_bytes(m, 'big')
    
    sigs = authsSigFors(sigFors)
    root = bytes()
    
    # roots calculation
    for i in range(0, K):
        
        #next index
        idx = (mInt >> (K -1 -i) * A) % T
        
        # leaf calculation
        sk = sigs[i][0]
        conn.setTreeHeight(0)
        conn.setTreeIndex(i * T + idx)
        nodo0 = hash(pkSeed, conn.copy(), sk, N)
        nodo1 = 0
        
        # root from authentication and leaf
        auth = sigs[i][1]
        conn.setTreeIndex(i * T + idx)
        
        for j in range(0, A):
            conn.setTreeHeight(j + 1)
        
            if math.floor(idx / 2 ** i) % 2 == 0:
                conn.setTreeIndex(conn.getTreeIndex() // 2)
                nodo1 = hash(pkSeed, conn.copy(), nodo0 + auth[j], N)
            else:
                conn.setTreeIndex((conn.getTreeIndex() - 1) // 2)
                nodo1 = hash(pkSeed, conn.copy(), auth[j] + nodo0, N)
            
            nodo0 = nodo1
            
        root += nodo1
        
    forsConn = conn.copy()
    forsConn.setType(Connect.FORSROOTS)
    forsConn.setPairAdrs(conn.getPairAdrs())
    
    pk = hash(pkSeed, forsConn, root, N)
    return pk

## SPHINCS+

    Aqui é implementado o algoritmo SPHINCS+ prorpiamente dito. ão utilizados todos os pontos anteriores para realizar a geração de chaves, assinatura e verificação.

In [9]:
# Usefull Methods

def sigWotsSigXmss(sig):
    return sig[0:L0]

def authSigXmss(sig):
    return sig[L0:]

def sigsXmssSigHt(sig):
    sigs = []
    for i in range(0, D):
        sigs.append(sig[i*(HPrime + L0):(i+1)*(HPrime + len_0)])
        
    return sigs

def authsSigFors(sig):
    sigs = []
    
    for i in range(0, K):
        sigs.append([])
        sigs[i].append(sig[(A + 1) * i])
        sigs[i].append(sig[((A+ 1) * i + 1):((A + 1) * (i + 1))])

    return sigs

# Key pair generation
def genKeyPair():
    
    sk, pk = shpKeygen()
    sk_0, pk_0 = bytes(), bytes()
    
    for i in sk:
        sk_0 += i
    
    for i in pk:
        pk_0 += i
        
    return sk_0, pk_0

# Key pair generation aux
def shpKeygen():
    
    skSeed = os.urandom(N)
    skPrf  = os.urandom(N)
    pkSeed = os.urandom(N)
    
    pkRoot = htGenPk(skSeed, pkSeed)
    
    return [skSeed, skPrf, pkSeed, pkRoot], [pkSeed, pkRoot]

# Signature 
def sign(m, sk):
    skTab = []
    
    for i in range(0, 4):
        skTab.append(sk[(i * N):((i + 1) * N)])
        
    sigTab = shpSign(m, skTab)
    
    sig = sigTab[0]
    
    for i in sigTab[1]:
        sig += i
    for i in sigTab[2]:
        sig += i

    return sig

# Signature aux
def shpSign(m, secretKey):
    conn = Connect()
    
    skSeed = secretKey[0]
    skPrf  = secretKey[1]
    pkSeed = secretKey[2]
    pkRoot = secretKey[3]
    
    opt = bytes(N)   
    if _randomize:
        opt = os.urandom(N)
    r = prfMsg(skPrf, opt, m, N)
    sig = [r]
    
    sizeMd = math.floor((K * A + 7) / 8)
    sizeIndexTree = math.floor((H - H // D + 7) / 8)
    sizeIndexLeaf = math.floor((H // D + 7) / 8)
    
    digest = hashMsg(r, pkSeed, pkRoot, m, sizeMd + sizeIndexTree + sizeIndexLeaf)
    tmpMd = digest[:sizeMd]
    tmpIndexTree = digest[sizeMd:(sizeMd + sizeIndexTree)]
    tmpIndexLeaf = digest[(sizeMd + sizeIndexTree):len(digest)]
    
    mdInt = int.from_bytes(tmpMd, 'big') >> (len(tmpMd) * 8 - K * A)
    md = mdInt.to_bytes(math.ceil(K * A / 8), 'big')
    
    indexTree = int.from_bytes(tmpIndexTree, 'big') >> (len(tmpIndexTree) * 8 - (H - H // D))
    indexLeaf = int.from_bytes(tmpIndexLeaf, 'big') >> (len(tmpIndexLeaf) * 8 - (H // D))
    
    conn.setLayerAdrs(0)
    conn.setTreeAdrs(indexTree)
    conn.setType(Connect.FORSTREE)
    conn.setPairAdrs(indexLeaf)
    
    sigFors = forsSign(md, skSeed, pkSeed, conn.copy())
    sig += [sigFors]
    
    pkFors = forsPkSig(sigFors, md, pkSeed, conn.copy())
    
    conn.setType(Connect.TREE)
    sigHt = htSing(pkFors, skSeed, pkSeed, indexTree, indexLeaf)
    sig += [sigHt]
    
    return sig

# Verify
def verify(m, sig, pk):
    pkTab = []
    
    for i in range(0, 2):
        pkTab.append(pk[(i * N):((i + 1) * N)])
        
    sigTab = []
    
    sigTab += [sig[:N]]
    sigTab += [[]]
    for i in range(N, N + K * (A + 1) * N, N):
        sigTab[1].append(sig[i:(i + N)])
        
    sigTab += [[]]
    for i in range(N + K * (A + 1) * N, N + K * (A + 1) * N + (H + D * L0) * N, N):
        sigTab[2].append(sig[i:(i + N)])
        
    return shpVerify(m, sigTab, pkTab)

# Verify aux
def shpVerify(m, sig, publicKey):
    conn = Connect()
    
    r = sig[0]
    sigFors = sig[1]
    sigHt = sig[2]
    
    pkSeed = publicKey[0]
    pkRoot = publicKey[1]
    
    sizeMd = math.floor((K * A + 7) / 8)
    sizeIndexTree = math.floor((H - H // D + 7) / 8)
    sizeIndexLeaf = math.floor((H // D + 7) / 8)
    
    digest = hashMsg(r, pkSeed, pkRoot, m, sizeMd + sizeIndexTree + sizeIndexLeaf)
    tmpMd = digest[:sizeMd]
    tmpIndexTree = digest[sizeMd:(sizeMd + sizeIndexTree)]
    tmpIndexLeaf = digest[(sizeMd + sizeIndexTree):len(digest)]
    
    mdInt = int.from_bytes(tmpMd, 'big') >> (len(tmpMd) * 8 - K * A)
    md = mdInt.to_bytes(math.ceil(K * A / 8), 'big')
    
    indexTree = int.from_bytes(tmpIndexTree, 'big') >> (len(tmpIndexTree) * 8 - (H - H // D))
    indexLeaf = int.from_bytes(tmpIndexLeaf, 'big') >> (len(tmpIndexLeaf) * 8 - (H // D))
    
    conn.setLayerAdrs(0)
    conn.setTreeAdrs(indexTree)
    conn.setType(Connect.FORSTREE)
    conn.setPairAdrs(indexLeaf)
    
    pkFors = forsPkSig(sigFors, md, pkSeed, conn)
    
    conn.setType(Connect.TREE)
    
    return htVerify(pkFors, sigHt, pkSeed, indexTree, indexTree, pkRoot)

## Teste

In [10]:
sk, pk = genKeyPair()

m = b'Test message'
print(m)

signature = sign(m, sk)
print(signature)

result = verify(m, signature, pk)
print(result)

b'Test message'
b'\xaa\tS\x8e\x1d\xa1"a\x048\x88\xc6z\xdc\xe8b}\x821m\xf7\xb7s}\xc5\x01\x1c\xc6s\xb3\xda\xe7\xf1\x8e\xa4\xf9\xbd\xde=\xc3\xc8\xab,G\x07%\xb6\xfe\xb5\xa2\xef\xceT\xa7G\xb7^r\x05\xe2\xb3\xbb|qQ\xb5861\x99\xd4\x8aS\x97\\\xbc\xfd\\\xfd\xf9\xaa\x86Q\x1e\x8b\xefx\xe4^\xb8\xe2\xa2Jn\x88\x10e]\xf60B\xce\xbf\x91(\xc1\xb1\x1cf\x16l\xe3\x08&8\xed\x9c@\x0ed\xcf\xfbC9/\x8d\x94\xc00\x03m\xe0\xf0\x14\x8b\x19tl\x1f\xce\\\xb3a\xcc\xe7\xcb\x1e\xf3y\xf8\xbdy\xb2\x8b\xe1\'&\x1b\xf7=\xe8\xed\x8a\x8c\xcf\xcd\xd3\xda\xc7?\xda\xf8U?\x9c\x84\x16\x16\x0b#\xc8\xca\x8b\x17\x05*0\xe16\xe3\x8e\x15\xac\xb0\x9cN\x1b\xb8\x907;\xa8\xdd\x95\x0bR\xd6\xc7h~|\xa3\x0bv\xd3{\x94\xed\xb1,\x82\xcb\xbb\xed\xde\xe2\x99\x9b\xa1&\xaf\xeb\xdd\xea\x86\xcf\x9a\xd2~k\x82\xe7.\xd8/9\x14J11[<;\x96\xfc\xbd\xb0P\xb1\x9a\xa3Q#\x1f\xda\t\xe1\x92\x0f\x04\x87O\x03"\xf5\xc7\x90\xab\xc7\xba\xeb\xcc\xf1\xe2\x83^\xc3\'\xaa\xf7P\xd1\x18d%4\xf4I\x15\x993\xc1\xdcOi~I\x8a\xb2 \xdfNEs\x9eg0(/\xb8\xd6\x7f\x0cB\n9\x89E\xee\xc2\x9e\x95CU\

MemoryError: 